# Subtypes

In [86]:
from collections import Counter

from rocksdict import Rdict

In [2]:
!ls

db2.rocks  db2.rocks.zip  subtypes.ipynb


In [3]:
rdict = Rdict('db2.rocks')

In [4]:
rdict['Q10011375']

{'instance_of': ['Q4167836']}

In [5]:
rdict['P10003']

{'instance_of': ['Q42396390', 'Q108914651']}

In [6]:
rdict['Q41176']

{'subclass_of': ['Q811979', 'Q13226383']}

In [7]:
%%time

entries_count = 0
for _ in rdict.keys():
    entries_count += 1
entries_count

CPU times: user 31.7 s, sys: 947 ms, total: 32.6 s
Wall time: 34.5 s


58661667

In [8]:
%%time

only_instance_of = 0
only_subclass_of = 0
both = 0
for key, values in rdict.items():
    if 'instance_of' in values and 'subclass_of' in values:
        both += 1
    elif 'instance_of' in values:
        only_instance_of += 1
    elif 'subclass_of' in values:
        only_subclass_of += 1

only_instance_of, only_subclass_of, both

CPU times: user 1min 33s, sys: 1.02 s, total: 1min 34s
Wall time: 1min 34s


(55912049, 350648, 2398970)

In [9]:
%%time

instance_of_lengths = Counter()
for key, values in rdict.items():
    if 'instance_of' in values:
        instance_of_lengths[len(values['instance_of'])] += 1

instance_of_lengths.most_common()

CPU times: user 1min 42s, sys: 840 ms, total: 1min 43s
Wall time: 1min 43s


[(1, 54582062),
 (2, 3248883),
 (3, 403636),
 (4, 54934),
 (5, 16645),
 (6, 3189),
 (7, 943),
 (8, 400),
 (9, 152),
 (10, 67),
 (11, 31),
 (12, 28),
 (13, 11),
 (15, 9),
 (14, 6),
 (16, 6),
 (24, 4),
 (20, 2),
 (18, 2),
 (92, 1),
 (51, 1),
 (58, 1),
 (25, 1),
 (40, 1),
 (19, 1),
 (55, 1),
 (27, 1),
 (22, 1)]

## NetworkX

In [10]:
import sys
import networkx as nx

In [11]:
%%time

subclass_edges = []
for key, values in rdict.items():
    for super_class in values.get('subclass_of', []):
        subclass_edges.append((key, super_class))
        
len(subclass_edges), sys.getsizeof(subclass_edges)

CPU times: user 1min 29s, sys: 1.13 s, total: 1min 31s
Wall time: 1min 31s


(3402319, 27436344)

In [12]:
subclass_graph = nx.DiGraph(subclass_edges)

In [13]:
nx.is_directed_acyclic_graph(subclass_graph)

False

In [14]:
nx.find_cycle(subclass_graph)

[('Q1799072', 'Q2695280'), ('Q2695280', 'Q1799072')]

In [15]:
# Empire State Building
rdict['Q9188']

{'instance_of': ['Q1021645', 'Q11303', 'Q570116']}

In [16]:
# skyscraper -> structure
%timeit nx.has_path(subclass_graph, 'Q11303', 'Q6671777')

33.7 µs ± 1.93 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [58]:
skipped = set()
double_instance = set()

In [60]:
def has_path_networkx_subclass(rdict: Rdict, source: str, target: str) -> bool:
    entries = rdict.get(source, dict()).get('instance_of', [])
    
    if target not in subclass_graph.nodes:
        skipped.add(target)
        return False
    
    double_instance.update(set(entries) - subclass_graph.nodes)
    
    return any([
        nx.has_path(subclass_graph, entry, target)
        for entry in set(entries) & subclass_graph.nodes
    ])

In [45]:
%timeit has_path_networkx_subclass(rdict, 'Q9188', 'Q6671777')

94.6 µs ± 1.37 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Using RocksDB

In [47]:
def has_path_rocksdb_subclass(rdict: Rdict, source: str, target: str) -> bool:
    visited = set()
    stack: list[str] = [source]
    visited.add(source)
    
    while stack:
        curr = stack.pop()
        for neigh in rdict[curr].get('subclass_of', []):
            if neigh in visited:
                continue
            
            if neigh == target:
                return True

            visited.add(neigh)
            stack.append(neigh)
    
    return False

In [37]:
%timeit has_path_rocksdb_subclass(rdict, 'Q11303', 'Q6671777')

178 µs ± 3.76 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [48]:
def has_path_rocksdb(rdict: Rdict, source: str, target: str) -> bool:
    entries = rdict[source].get('instance_of', [])
    return any([has_path_rocksdb_subclass(rdict, entry, target) for entry in entries])

In [49]:
%timeit has_path_rocksdb(rdict, 'Q9188', 'Q6671777')

381 µs ± 11 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## All NetworkX

Do not try this at home! :)

In [ ]:
# %%time

# all_edges = []
# for key, values in rdict.items():
#     for super_class in values.get('subclass_of', []) + values.get('instance_of', []):
#         all_edges.append((key, super_class))
        
# len(all_edges), sys.getsizeof(all_edges)

In [ ]:
all_graph = nx.DiGraph(all_edges)

# Benchmark

In [20]:
!ls

db2.rocks      subtypes.ipynb	       types_to_validate.zip
db2.rocks.zip  types_to_validate.json


In [27]:
import json, tqdm

In [21]:
with open('types_to_validate.json', 'r', encoding='utf-8') as f:
    types_to_validate = json.load(f)

In [78]:
with open('f', 'r', encoding='utf-8') as f:
    q5_correct = json.load(f)

In [22]:
len(types_to_validate)

8594

In [25]:
list(types_to_validate.keys())[:5]

['http://www.wikidata.org/entity/Q5',
 'http://www.wikidata.org/entity/Q4',
 'http://www.wikidata.org/entity/Q571',
 'http://www.wikidata.org/entity/Q7366',
 'http://www.wikidata.org/entity/Q7278']

In [26]:
len(types_to_validate['http://www.wikidata.org/entity/Q5'])

62692

In [61]:
%%time

validated = dict()
for target_url, sources in tqdm.tqdm(list(types_to_validate.items())):
    target = target_url.rsplit('/')[-1]
    validated[target] = []
    for source in sources:
        if has_path_networkx_subclass(rdict, source, target):
            validated[target].append(source)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8594/8594 [01:02<00:00, 137.14it/s]

CPU times: user 59.7 s, sys: 2.96 s, total: 1min 2s
Wall time: 1min 2s


In [106]:
len(validated['Q5'])  # from types_to_validate.json

43

In [104]:
%%time
for candidate in q5_correct['correct']:
    assert nx.has_path(subclass_graph, candidate, 'Q5')

for candidate in q5_correct['incorrect']:
    if candidate not in subclass_graph:
        continue
    
    if nx.has_path(subclass_graph, candidate, 'Q5'):
        print(candidate)
#     assert not nx.has_path(subclass_graph, candidate, 'Q5'), candidate

Q172964
CPU times: user 5.53 s, sys: 0 ns, total: 5.53 s
Wall time: 5.53 s


In [105]:
len(q5_correct['correct'])

350